In [1]:
import numpy as np
from deck import full_euchre_deck
from branch_calc import (
    filter_branch_by_hand,
    setdiff2d_idx,
    suit_id,
    find_winner,
    calc_all_possible_hands,
)
from dealer import dealer

In [2]:
game = dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands = [game.hand1, game.hand2, game.hand3, game.hand4]
first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

# all playable hands
all_possible_tricks = calc_all_possible_hands(hands=hands)
len(all_possible_tricks)

branches = {}
i = 0
# creates 5 branches with 125 possible tricks each
for card in first_play:
    i += 1
    contains_target = np.any(np.all(all_possible_tricks == card, axis=-1), axis=1)
    branches[f'branch{i}'] = all_possible_tricks[contains_target]
    # creates all possible tricks based on rules for following suit
i=-1
for branch in branches.keys():
    i+=1
    branches[branch] = filter_branch_by_hand(branches[branch] , game.hand2, 1, first_play[i])
    branches[branch] = filter_branch_by_hand(branches[branch], game.hand3, 2, first_play[i])
    branches[branch] = filter_branch_by_hand(branches[branch], game.hand4, 3, first_play[i])

In [21]:
len(branches['branch1']), len(branches['branch2']), len(branches['branch3']), len(branches['branch4']), len(branches['branch5'])

(2, 5, 5, 5, 5)

In [22]:
branches['branch1']

array([[[-12,   0],
        [-14,   0],
        [ -9,   0],
        [-11,   0]],

       [[-12,   0],
        [-10,   0],
        [ -9,   0],
        [-11,   0]]])

In [23]:
rd1_results = []
rd2_lead = []
for trick in branches['branch1']:
    rd2_winner = find_winner(lead=trick[0], trick=trick) 
    winning_team = find_winner(lead=trick[0], trick=trick) % 2 # binary results for partners 1,3 and 2,4
    rd1_results.append(winning_team)
    rd2_lead.append(rd2_winner)

rd1_results, rd2_lead

([1, 0], [1, 0])

In [24]:
rd2_all_possible_tricks = []
for trick in branches['branch1']:
    rd2_hands = [setdiff2d_idx(game.hand1, trick),
    setdiff2d_idx(game.hand2, trick),
    setdiff2d_idx(game.hand3, trick),
    setdiff2d_idx(game.hand4, trick)]
    rd2_all_possible_tricks.append(calc_all_possible_hands(hands=rd2_hands))

rd2_ap_arr = np.array(rd2_all_possible_tricks)

rd2_lead_hands = []
for player in rd2_lead:
    first_play = [rd2_hands[player][0], rd2_hands[player][1], rd2_hands[player][2], rd2_hands[player][3]]
    rd2_lead_hands.append(first_play)

In [36]:

for hand in rd2_lead_hands:
    print(hand)
    for card in hand:
        print(card)

[array([  0, -13]), array([  0, 130]), array([-14,   0]), array([ 0, -9])]
[  0 -13]
[  0 130]
[-14   0]
[ 0 -9]
[array([  0, 110]), array([  0, 140]), array([ 0, 90]), array([  0, 100])]
[  0 110]
[  0 140]
[ 0 90]
[  0 100]


In [37]:
for leader in rd2_lead:
    print(leader)

1
0


In [39]:
rd2_grids = []
for leader in rd2_lead:
    for hand in rd2_lead_hands:
        print(hand)
        for card in hand:
            print(card)
            grid = rd2_ap_arr[leader]
            rd2_contains_target = np.any(np.all(grid == card, axis=-1), axis=1)
            rd2_grids.append(grid[rd2_contains_target])
            

[array([  0, -13]), array([  0, 130]), array([-14,   0]), array([ 0, -9])]
[  0 -13]
[  0 130]
[-14   0]
[ 0 -9]
[array([  0, 110]), array([  0, 140]), array([ 0, 90]), array([  0, 100])]
[  0 110]
[  0 140]
[ 0 90]
[  0 100]
[array([  0, -13]), array([  0, 130]), array([-14,   0]), array([ 0, -9])]
[  0 -13]
[  0 130]
[-14   0]
[ 0 -9]
[array([  0, 110]), array([  0, 140]), array([ 0, 90]), array([  0, 100])]
[  0 110]
[  0 140]
[ 0 90]
[  0 100]


In [40]:
len(rd2_grids)

16

In [28]:
# # gotta fix this block to account for different players starting the game. 
# for j in np.arange(1, 4*len(branches['branch1'])+1):
#     branches[f'branch1_{j}'] = filter_branch_by_hand(branches[f'branch1_{j}'] , game.hand2, 1, card)
#     branches[f'branch1_{j}'] = filter_branch_by_hand(branches[f'branch1_{j}'], game.hand3, 2, card)
#     branches[f'branch1_{j}'] = filter_branch_by_hand(branches[f'branch1_{j}'], game.hand4, 3, card)